# European Call and Put Option Pricing
In this notebook, we calculate prices of European call and put options using Black-Scholes formula and the Monte Carlo method.

We assume that the stock prices follow Geometric Brownian Motion, i.e. the price $S_t$ at time $t$ of a stock can be expressed as
$$
S_t = S_0 \exp \left\{ \left( r - \frac{1}{2} \sigma^2 \right) t + \sigma W_t \right\}
$$
where $S_0$ is the initial stock price (at time 0), $r$ is the risk-free interest rate, $\sigma$ is the volatility of the stock, and $W_t$ is the standard Brownian motion.

In [10]:
import numpy as np
import pandas as pd
from scipy.stats import norm

print('numpy version: ',np.__version__)
print('pandas version: ',pd.__version__)

numpy version:  1.17.4
pandas version:  0.25.3


### Black-Scholes Formula for European Call and Put Options

We denote the maturity by $T$ and the strike price by $K$. Then the Black-Scholes price of the option is given by
$$
\mathbb{E}[(K - S_T)^+] = S_0 \Phi(\sigma \sqrt T - \theta) - K e^{-rT} \Phi(-\theta) \quad \text{where} \quad 
\theta = \frac{1}{\sigma \sqrt T} \log \frac{K}{S_0} + \left( \frac{1}{2} \sigma - \frac{r}{\sigma} \right) \sqrt T
$$
and $\Phi$ is the $CDF$ of the standard normal.


In [2]:
def BS_call(S_0, r, sigma, K, T):
    '''
    Black-Scholes price for European call option:
    S_0 - price of the underlying stock at time 0
    r - annual risk-free interest rate in decimal
    sigma - volatility of the underlying stock
    K - strike price
    T - time to maturity in years
    The result is rounded to 5 decimal places
    '''
    theta = (1 / (sigma * np.sqrt(T))) * np.log(K / S_0) + (0.5 * sigma - r / sigma) * np.sqrt(T)
    bs_call_price = S_0 * norm.cdf(sigma * np.sqrt(T) - theta) - K * np.exp(-r * T) * norm.cdf(-theta)
    return round(bs_call_price, 5)

We calculate the price of an European put option with the same parameters using the 'put-call parity':
$$
\mathbb{E}[(S_T - K)^+] = \mathbb{E}[(K - S_T)^+] - S_0 + K e^{-rT}
$$


In [3]:
def BS_put(S_0, r, sigma, K, T):
    '''
    Black-Scholes price for European put option:
    S_0 - price of the underlying stock at time 0
    r - annual risk-free interest rate in decimal
    sigma - volatility of the underlying stock
    K - strike price
    T - time to maturity in years
    The result is rounded to 5 decimal places
    '''
    bs_put_price = BS_call(S_0, r, sigma, K, T) - S_0 + K * np.exp(-r * T)
    return round(bs_put_price, 5)

### Calculation of European Call and Put Options Using Monte Carlo

In [4]:
def MC_call(S_0, r, sigma, K, T, n):
    '''
    Price for European call option using Monte Carlo simulation:
    S_0 - price of the underlying stock at time 0
    r - annual risk-free interest rate in decimal
    sigma - volatility of the underlying stock
    K - strike price
    T - time to maturity in years
    n - number of iterations
    Returns both the price and standard error
    The results are  rounded to 5 decimal places
    '''
    Z = np.random.standard_normal(size=n)
    Y = S_0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    X = np.exp(-r * T) * np.maximum(Y - K, np.zeros(n))
    mc_call_price = np.mean(X)
    mc_call_se = np.std(X) / np.sqrt(n)
    return round(mc_call_price, 5), round(mc_call_se, 5)

In [5]:
def MC_put(S_0, r, sigma, K, T, n):
    '''
    Price for European put option using Monte Carlo simulation:
    S_0 - price of the underlying stock at time 0
    r - annual risk-free interest rate in decimal
    sigma - volatility of the underlying stock
    K - strike price
    T - time to maturity in years
    n - number of iterations
    Returns both the price and standard error
    The results are  rounded to 5 decimal places
    '''
    Z = np.random.standard_normal(size=n)
    Y = S_0 * np.exp((r - 0.5 * sigma ** 2) * T + sigma * np.sqrt(T) * Z)
    X = np.exp(-r * T) * np.maximum(K - Y, np.zeros(n))
    mc_put_price = np.mean(X)
    mc_put_se = np.std(X) / np.sqrt(n)
    return round(mc_put_price, 5), round(mc_put_se, 5)

### Comparison of the Black-Scholes Price and Monte Carlo Estimate

We now compare the Black-Scholes price of call and put options and their Monte Carlo estimates for various strike prices and various sample sizes. 

#### European Call Option with strike price $K=40$,  $S_0 = 50$, $r=0.05$, $\sigma=0.2$, and $T=1$.

In [6]:
np.random.seed(2019)
sample_size = [1e4, 1e5, 1e6, 1e7, 1e8]
mc_estimates = []
mc_se = []
bs_price = np.repeat(BS_call(50, 0.05, 0.2, 40, 1), 5)

for n in sample_size:
    mc_call = MC_call(50, 0.05, 0.2, 40, 1, int(n))
    mc_estimates.append(mc_call[0])
    mc_se.append(mc_call[1])

df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'Monte Carlo Estimate': mc_estimates, 'Standard Error': mc_se})
df

,Sample Size,Black-Scholes Price,Monte Carlo Estimate,Standard Error
0,10000.0,12.29442,12.27792,0.09577
1,100000.0,12.29442,12.33168,0.03035
2,1000000.0,12.29442,12.29618,0.00958
3,10000000.0,12.29442,12.29345,0.00303
4,100000000.0,12.29442,12.29350,0.00096


#### European Call Option with strike price $K=60$,  $S_0 = 50$, $r=0.05$, $\sigma=0.2$, and $T=1$.

In [7]:
np.random.seed(2019)
sample_size = [1e4, 1e5, 1e6, 1e7, 1e8]
mc_estimates = []
mc_se = []
bs_price = np.repeat(BS_call(50, 0.05, 0.2, 60, 1), 5)

for n in sample_size:
    mc_call = MC_call(50, 0.05, 0.2, 60, 1, int(n))
    mc_estimates.append(mc_call[0])
    mc_se.append(mc_call[1])

df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'Monte Carlo Estimate': mc_estimates, 'Standard Error': mc_se})
df

,Sample Size,Black-Scholes Price,Monte Carlo Estimate,Standard Error
0,10000.0,1.62374,1.62142,0.04292
1,100000.0,1.62374,1.64203,0.01374
2,1000000.0,1.62374,1.62443,0.00434
3,10000000.0,1.62374,1.62476,0.00137
4,100000000.0,1.62374,1.62307,0.00043


#### European Put Option with strike price $K=40$,  $S_0 = 50$, $r=0.05$, $\sigma=0.2$, and $T=1$.

In [8]:
np.random.seed(2019)
sample_size = [1e4, 1e5, 1e6, 1e7, 1e8]
mc_estimates = []
mc_se = []
bs_price = np.repeat(BS_put(50, 0.05, 0.2, 40, 1), 5)

for n in sample_size:
    mc_put = MC_put(50, 0.05, 0.2, 40, 1, int(n))
    mc_estimates.append(mc_put[0])
    mc_se.append(mc_put[1])

df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'Monte Carlo Estimate': mc_estimates, 'Standard Error': mc_se})
df

,Sample Size,Black-Scholes Price,Monte Carlo Estimate,Standard Error
0,10000.0,0.3436,0.35128,0.01354
1,100000.0,0.3436,0.34180,0.00421
2,1000000.0,0.3436,0.34300,0.00134
3,10000000.0,0.3436,0.34359,0.00042
4,100000000.0,0.3436,0.34368,0.00013


#### European Put Option with strike price $K=60$,  $S_0 = 50$, $r=0.05$, $\sigma=0.2$, and $T=1$.

In [9]:
np.random.seed(2019)
sample_size = [1e4, 1e5, 1e6, 1e7, 1e8]
mc_estimates = []
mc_se = []
bs_price = np.repeat(BS_put(50, 0.05, 0.2, 60, 1), 5)

for n in sample_size:
    mc_put = MC_put(50, 0.05, 0.2, 60, 1, int(n))
    mc_estimates.append(mc_put[0])
    mc_se.append(mc_put[1])

df = pd.DataFrame({'Sample Size': sample_size, 'Black-Scholes Price': bs_price, 
                   'Monte Carlo Estimate': mc_estimates, 'Standard Error': mc_se})
df

,Sample Size,Black-Scholes Price,Monte Carlo Estimate,Standard Error
0,10000.0,8.69751,8.71936,0.07449
1,100000.0,8.69751,8.67674,0.02344
2,1000000.0,8.69751,8.69585,0.00741
3,10000000.0,8.69751,8.69949,0.00235
4,100000000.0,8.69751,8.69784,0.00074


#### Observation:
If one increases the sample size in a Monte Carlo estimation by a factor of $n$, the standard error will be reduced by a factor of $\sqrt n$. One can easily observe this phenomennon by comparing the 1st and the 3rd rows (or the 3rd and 5th rows for that matter) in each dataframe above by noting the 100-fold increase in sample size and 10-fold decrease in standard error.